In [1]:
import pandas as pd
import numpy as np
from gtfparse import read_gtf
import functools as ft
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read GTF with gene annotation
genes = read_gtf('/DATA/users/magnitov/znf143/genome/gencode.vM25.annotation.gtf')
genes = genes[genes['feature'] == 'gene']
genes = genes[['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name']]

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


## Genes

In [5]:
def get_counts_table(genes, sample, replicate):
    counts_fwd = pd.read_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_forward.ZFP143_TTseq.txt', 
                         names = ['gene_id', 'Zfp143_DMSO_rep1', 'Zfp143_DMSO_rep2', 'Zfp143_2H_rep1', 'Zfp143_2H_rep2', 
                                  'Zfp143_6H_rep1', 'Zfp143_6H_rep2', 'Zfp143_24H_rep1', 'Zfp143_24H_rep2'],
                         sep = '\t', header = None)
    counts_fwd = counts_fwd[counts_fwd['gene_id'].isin([x for x in counts_fwd['gene_id'] if x[0]!='_'])]
    counts_fwd = counts_fwd[['gene_id', sample + '_' + replicate]]
    
    counts_rev = pd.read_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_reverse.ZFP143_TTseq.txt', 
                         names = ['gene_id', 'Zfp143_DMSO_rep1', 'Zfp143_DMSO_rep2', 'Zfp143_2H_rep1', 'Zfp143_2H_rep2', 
                                  'Zfp143_6H_rep1', 'Zfp143_6H_rep2', 'Zfp143_24H_rep1', 'Zfp143_24H_rep2'],
                         sep = '\t', header = None)
    counts_rev = counts_rev[counts_rev['gene_id'].isin([x for x in counts_rev['gene_id'] if x[0]!='_'])]
    counts_rev = counts_rev[['gene_id', sample + '_' + replicate]]
    
    genes_merged = genes.merge(counts_fwd, on = 'gene_id', how = 'left')
    genes_merged = genes_merged.merge(counts_rev, on = 'gene_id', how = 'left')
    
    counts_new = []
    for gene in genes_merged.values:
        if gene[3] == '+':
            counts_new.append(gene[7])
        else:
            counts_new.append(gene[8])
    genes_merged[f'{sample}_{replicate}'] = counts_new
    
    genes_merged = genes_merged.fillna(0)
    
    genes_merged = genes_merged[['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name', f'{sample}_{replicate}']]
    
    return(genes_merged)

In [6]:
counts_Zfp143_DMSO_rep1 = get_counts_table(genes, 'Zfp143_DMSO', 'rep1')
counts_Zfp143_DMSO_rep2 = get_counts_table(genes, 'Zfp143_DMSO', 'rep2')
counts_Zfp143_2H_rep1 = get_counts_table(genes, 'Zfp143_2H', 'rep1')
counts_Zfp143_2H_rep2 = get_counts_table(genes, 'Zfp143_2H', 'rep2')
counts_Zfp143_6H_rep1 = get_counts_table(genes, 'Zfp143_6H', 'rep1')
counts_Zfp143_6H_rep2 = get_counts_table(genes, 'Zfp143_6H', 'rep2')
counts_Zfp143_24H_rep1 = get_counts_table(genes, 'Zfp143_24H', 'rep1')
counts_Zfp143_24H_rep2 = get_counts_table(genes, 'Zfp143_24H', 'rep2')

count_tables = [counts_Zfp143_DMSO_rep1, counts_Zfp143_DMSO_rep2, counts_Zfp143_2H_rep1, counts_Zfp143_2H_rep2, 
                counts_Zfp143_6H_rep1, counts_Zfp143_6H_rep2, counts_Zfp143_24H_rep1, counts_Zfp143_24H_rep2]
counts_zfp143 = ft.reduce(lambda left, right: pd.merge(left, right, on = ['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name']), count_tables)
for column in counts_zfp143.columns[7:]:
    counts_zfp143[column] = [int(x) for x in counts_zfp143[column]]
counts_zfp143.head()

,seqname,start,end,strand,gene_id,gene_type,gene_name,Zfp143_DMSO_rep1,Zfp143_DMSO_rep2,Zfp143_2H_rep1,Zfp143_2H_rep2,Zfp143_6H_rep1,Zfp143_6H_rep2,Zfp143_24H_rep1,Zfp143_24H_rep2
0,chr1,3073253,3074322,+,ENSMUSG00000102693.1,TEC,4933401J01Rik,0,0,0,0,0,0,0,0
1,chr1,3102016,3102125,+,ENSMUSG00000064842.1,snRNA,Gm26206,0,0,0,0,0,0,0,0
2,chr1,3205901,3671498,-,ENSMUSG00000051951.5,protein_coding,Xkr4,2,17,11,13,3,10,8,16
3,chr1,3252757,3253236,+,ENSMUSG00000102851.1,processed_pseudogene,Gm18956,0,0,0,0,0,0,0,0
4,chr1,3365731,3368549,-,ENSMUSG00000103377.1,TEC,Gm37180,0,0,0,0,0,0,0,0


In [7]:
counts_zfp143.drop(['seqname', 'start', 'end', 'strand', 'gene_type', 'gene_name'], axis = 1).to_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_deseq2.txt', sep = '\t', header = 0, index = 0)
counts_zfp143.to_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_analysis.txt', sep = '\t', header = 1, index = 0)

## Transcripts

In [12]:
# Read GTF with gene annotation
genes = read_gtf('/DATA/users/magnitov/znf143/genome/gencode.vM25.annotation.gtf')
genes = genes[genes['feature'] == 'transcript']
genes = genes[['seqname', 'start', 'end', 'strand', 'transcript_id', 'transcript_type', 'gene_name']]

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


In [13]:
def get_counts_table(genes, sample, replicate):
    counts_fwd = pd.read_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_transcripts_forward.ZFP143_TTseq.txt', 
                         names = ['transcript_id', 'Zfp143_DMSO_rep1', 'Zfp143_DMSO_rep2', 'Zfp143_2H_rep1', 'Zfp143_2H_rep2', 
                                  'Zfp143_6H_rep1', 'Zfp143_6H_rep2', 'Zfp143_24H_rep1', 'Zfp143_24H_rep2'],
                         sep = '\t', header = None)
    counts_fwd = counts_fwd[counts_fwd['transcript_id'].isin([x for x in counts_fwd['transcript_id'] if x[0]!='_'])]
    counts_fwd = counts_fwd[['transcript_id', sample + '_' + replicate]]
    
    counts_rev = pd.read_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_transcripts_reverse.ZFP143_TTseq.txt', 
                         names = ['transcript_id', 'Zfp143_DMSO_rep1', 'Zfp143_DMSO_rep2', 'Zfp143_2H_rep1', 'Zfp143_2H_rep2', 
                                  'Zfp143_6H_rep1', 'Zfp143_6H_rep2', 'Zfp143_24H_rep1', 'Zfp143_24H_rep2'],
                         sep = '\t', header = None)
    counts_rev = counts_rev[counts_rev['transcript_id'].isin([x for x in counts_rev['transcript_id'] if x[0]!='_'])]
    counts_rev = counts_rev[['transcript_id', sample + '_' + replicate]]
    
    genes_merged = genes.merge(counts_fwd, on = 'transcript_id', how = 'left')
    genes_merged = genes_merged.merge(counts_rev, on = 'transcript_id', how = 'left')
    
    counts_new = []
    for gene in genes_merged.values:
        if gene[3] == '+':
            counts_new.append(gene[7])
        else:
            counts_new.append(gene[8])
    genes_merged[f'{sample}_{replicate}'] = counts_new
    
    genes_merged = genes_merged.fillna(0)
    
    genes_merged = genes_merged[['seqname', 'start', 'end', 'strand', 'transcript_id', 'transcript_type', 'gene_name', f'{sample}_{replicate}']]
    
    return(genes_merged)

In [14]:
counts_Zfp143_DMSO_rep1 = get_counts_table(genes, 'Zfp143_DMSO', 'rep1')
counts_Zfp143_DMSO_rep2 = get_counts_table(genes, 'Zfp143_DMSO', 'rep2')
counts_Zfp143_2H_rep1 = get_counts_table(genes, 'Zfp143_2H', 'rep1')
counts_Zfp143_2H_rep2 = get_counts_table(genes, 'Zfp143_2H', 'rep2')
counts_Zfp143_6H_rep1 = get_counts_table(genes, 'Zfp143_6H', 'rep1')
counts_Zfp143_6H_rep2 = get_counts_table(genes, 'Zfp143_6H', 'rep2')
counts_Zfp143_24H_rep1 = get_counts_table(genes, 'Zfp143_24H', 'rep1')
counts_Zfp143_24H_rep2 = get_counts_table(genes, 'Zfp143_24H', 'rep2')

count_tables = [counts_Zfp143_DMSO_rep1, counts_Zfp143_DMSO_rep2, counts_Zfp143_2H_rep1, counts_Zfp143_2H_rep2, 
                counts_Zfp143_6H_rep1, counts_Zfp143_6H_rep2, counts_Zfp143_24H_rep1, counts_Zfp143_24H_rep2]
counts_zfp143 = ft.reduce(lambda left, right: pd.merge(left, right, on = ['seqname', 'start', 'end', 'strand', 'transcript_id', 'transcript_type', 'gene_name']), count_tables)
for column in counts_zfp143.columns[7:]:
    counts_zfp143[column] = [int(x) for x in counts_zfp143[column]]
counts_zfp143.head()

,seqname,start,end,strand,transcript_id,transcript_type,gene_name,Zfp143_DMSO_rep1,Zfp143_DMSO_rep2,Zfp143_2H_rep1,Zfp143_2H_rep2,Zfp143_6H_rep1,Zfp143_6H_rep2,Zfp143_24H_rep1,Zfp143_24H_rep2
0,chr1,3073253,3074322,+,ENSMUST00000193812.1,TEC,4933401J01Rik,0,0,0,0,0,0,0,0
1,chr1,3102016,3102125,+,ENSMUST00000082908.1,snRNA,Gm26206,0,0,0,0,0,0,0,0
2,chr1,3205901,3216344,-,ENSMUST00000162897.1,processed_transcript,Xkr4,0,0,0,0,0,0,0,0
3,chr1,3206523,3215632,-,ENSMUST00000159265.1,processed_transcript,Xkr4,0,0,0,0,0,0,0,0
4,chr1,3214482,3671498,-,ENSMUST00000070533.4,protein_coding,Xkr4,2,17,11,13,3,10,8,16


In [15]:
counts_zfp143.drop(['seqname', 'start', 'end', 'strand', 'transcript_type', 'gene_name'], axis = 1).to_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_transcripts_deseq2.txt', sep = '\t', header = 0, index = 0)
counts_zfp143.to_csv('/DATA/users/magnitov/znf143/ttseq/counts/counts_transcripts_analysis.txt', sep = '\t', header = 1, index = 0)